In [1]:
file_path = '/Users/shiveshkodali/Desktop/GitProjects/SentimentAnalysis/'

import numpy as np
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data = pd.read_csv(file_path+'Data/train.tsv',sep='\t')

In [3]:
documents = list(data['Phrase'])
labels = list(data['Sentiment'])

In [4]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

In [5]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
!pip install xgboost

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# Create and train different machine learning models
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes" : MultinomialNB()
}

def sentiment_analysis(model_name):
    model = models[model_name]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    # Display results
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy:.2f}\n")
    print(report)
    print("=" * 50)
    return

In [9]:
sentiment_analysis('Logistic Regression')

Model: Logistic Regression
Accuracy: 0.65

              precision    recall  f1-score   support

           0       0.55      0.29      0.38      1416
           1       0.56      0.40      0.46      5527
           2       0.69      0.88      0.77     15639
           3       0.59      0.47      0.53      6707
           4       0.57      0.35      0.43      1923

    accuracy                           0.65     31212
   macro avg       0.59      0.48      0.51     31212
weighted avg       0.63      0.65      0.62     31212



/Users/shiveshkodali/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
sentiment_analysis('Decision Tree')

In [ ]:
sentiment_analysis('Naive Bayes')